In [ ]:
topics = ['Боли в спине (пояснице, шее, между лопатками): ~15%',
'Проблемы с ЖКТ (гастрит, рефлюкс, вздутие, газообразование, запор, диарея): ~15%',
'Проблемы с суставами (артрит, артроз, хруст, боль): ~10%',
'Головные боли (мигрень, хронические): ~8%',
'Проблемы с кожей (псориаз, экзема, дерматит, прыщи, бородавки, папилломы): ~8%',
'Усталость, слабость, потеря энергии: ~7%',
'Проблемы с сердцем (тахикардия, перебои, боль): ~5%',
'Проблемы с ногами (боль, онемение, судороги, отеки): ~5%',
'Проблемы с весом (лишний вес, набор веса): ~4%',
'Проблемы с давлением (высокое, низкое): ~4%',
'Проблемы с дыханием (астма, кашель, одышка): ~3%',
'Проблемы с щитовидной железой: ~3%',
'Проблемы с желчным пузырем (камни, полипы, перегиб): ~3%',
'Проблемы с мочеполовой системой: ~2%',
'Проблемы с глазами: ~2%',
'Проблемы с волосами (выпадение, седина): ~2%',
'Психические проблемы (тревога, ПА, депрессия): ~2%',
'Проблемы с иммунитетом: ~1%',
'Проблемы с лимфоузлами: ~1%',
'Проблемы с обменом веществ: ~1%']



In [2]:
# Define the path to your text file
file_path = 'questions.txt'

# Initialize an empty list to hold the final questions
final_questions = []

# Step 1: Read the .txt file and process it line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Step 2: Check if the line contains [ or ] and skip it if so
        if '[' in line or ']' in line:
            continue
        
        # Since we're reading line by line, each line ends with a '\n', except possibly the last one
        # Splitting by '\n' here is implicit in the reading process, so we don't need an explicit split operation
        
        # Step 4: Add the cleaned line to the final questions list
        # Trim any leading/trailing whitespace that might affect the cleanup
        cleaned_line = line.strip()
        
        # Ensure the line is not empty after stripping
        if cleaned_line:
            final_questions.append(cleaned_line)

final_questions

['Что делать - псориаз',
 'что делать если болит спина в пояснице',
 'Как растворить камень желчном пузыре 1.6 см',
 'Что делать если очень сильно повышено газообразование , буквально газовая бомба😁',
 'Что делать, если артрит колена и тромбоз, а нужно двигаться?',
 'Что делать, если в животе в положении лежа выше пупка бывает жжение, урчание и вздутие?',
 'что делать, если ревматоидный артрит 77 лет?',
 'Что делать если высокий холестерин и триглицериды у шестилетнего ребёнка',
 'Что делать если  мерзнут ноги даже дома в тепле?',
 'Что делать, если хрустят все суставы.',
 'Что делать если рацион питания не менялся а вес увеличился и появились прыщи на теле',
 'Что делать ? Сильные головные боли - мигрень?',
 'Что делать,если атрофия мышц,59 лет',
 'Что делать, если температура тела упала до 35.7? О чем это говорит?',
 'Что делать, если у ребенка сахарный диабет первого типа с 4,5 лет, сейчас 8 лет. И повышенный холестерин.',
 'Что делать, если полип в желчном пузыре?',
 'Что делать, е

In [52]:
clusters = {
'Проблемы с пищеварением': [],
'Боли в суставах и мышцах': [],
'Головные боли и мигрень': [],
'Проблемы с кожей': [],
'Психологические проблемы': [],
'Проблемы с сердцем и сосудами': [],
'Проблемы с эндокринной системой': [],
'Проблемы с дыхательной системой': [],
'Проблемы с мочеполовой системой': [],
'Вопросы о долголетии и общем здоровье': [],
'Проблемы с глазами': [],
'Проблемы с ушами': [],
'Проблемы с ногами': [],
'Проблемы с руками': [],
'Проблемы с кожей головы и волосами': [],
'Проблемы с зубами и ртом': [],
'Проблемы с температурой тела': [],
'Проблемы с лимфатической системой': [],
'Проблемы с весом': [],
'Прочие вопросы:': []
}

prompt = """
Вам представлен список вопросов.
Вот список вопросов:

<questions>
{$QUESTIONS}
</questions>
И вот список существующих кластеров:

<clusters>
{$CLUSTERS}
</clusters>
Ваша задача - категоризировать каждый вопрос в один из существующих кластеров, если это возможно. Если подходящего существующего кластера для вопроса нет, создайте для него новый кластер.

Чтобы выполнить эту задачу:

Внимательно прочитайте список вопросов, чтобы понять, что каждый из них спрашивает.
Ознакомьтесь со списком названий и описаний существующих кластеров, чтобы понять, какие темы они охватывают.
Для каждого вопроса определите, в какой существующий кластер он лучше всего подходит, исходя из предмета. Если подходящего существующего кластера нет, создайте новое название кластера для вопроса.
После категоризации всех вопросов выведите свои результаты в формате JSON со следующей структурой:

{
"вопрос 1": "название кластера",
"вопрос 2": "назавние кластера",
...
}

Выведите результат в формате json
"""

from openai import OpenAI
import json

def llm_api_call(history):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4-turbo-preview",
        messages=history,
        temperature=0.6,
        # max_tokens=1215,
        top_p=0.49,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={ "type": "json_object" }
        )
    return response

questions_dict = dict(zip(range(len(final_questions)), final_questions))
start = 0
step = 30


30 30


In [53]:
for end in range(step, len(final_questions)+step, step)[2:]:
    questions = final_questions[start:end]

    start = end
    prompt_filled = prompt.replace("$QUESTIONS", '/n'.join(questions)).replace("$CLUSTERS", '/n'.join(list(clusters.keys())))
    
    history=[
        {
        "role": "user",
        "content": prompt_filled
        }]
    response = llm_api_call(history)
    final_answer = response.choices[0].message.content

    q2k = json.loads(final_answer)
    print(end, len(q2k))

    for key, value in q2k.items():
        if value not in clusters:
            clusters[value] = []
        clusters[value].append(key)

    clusters


90 57
120 29
150 30
180 28
210 26
240 29
270 30
300 25
330 30
360 25
390 27
420 10


In [59]:
with open('clusters.json', 'w', ) as f:
    json.dump(clusters, f, indent=4, ensure_ascii=False)


In [ ]:
Топ-20 вопросов о проблемах с пищеварением:
Что делать, если в животе в положении лежа выше пупка бывает жжение, урчание и вздутие?
Что делать, если в кишечнике постоянное урчание-бурчание и повышенное газообразование?
Что делать, если все время дует кишечник и болит?
Что делать, если от рефлюкса жжет язык, нёбо и в горле?
Что делать, если есть перегиб желчного пузыря и дискинезия желчевыводящих путей?
Что делать, если после приема клетчатки и правильного, сбалансированного питания, сразу же зловонное выделение газов и опорожнение кишечника и так после каждой еды?
Что делать, если болит справа под ребрами и высокий билирубин, а так же холестерин?
Что делать, если язык желтый и вяжет во рту?
Что делать, если есть забросы желчи в пищевод?
Что делать, если после еды в желудке как будто вулкан? Начинается сильная отрыжка воздухом, а в животе бурлит река?
Что делать, если есть грыжа пищевода?
Что делать, если горечь во рту, хронический холецистит, хронический эрозивный гастрит, панкреатит?
Что делать, если пониженная кислотность, загиб желчного пузыря, грибок Кандидо, дисбактериоз?
Что делать, если нефункционирующий (отключенный) желчный пузырь?
Что делать, если есть гастродуоденит, тошнота по утрам?
Что делать, если отрыжка воздухом постоянная, слизь в горле, кашель, вздутие живота?
Что делать, если болит живот?
Что делать, если есть запор?
Что делать, если есть диарея?
Что делать, если есть изжога?
Важно отметить, что эти вопросы не обязательно являются самыми распространенными проблемами с пищеварением, но они были заданы людьми, которые ищут информацию и помощь.
